[https://mp.weixin.qq.com/s/VIQIyg_aWSKx--syBwMLdg](https://mp.weixin.qq.com/s/VIQIyg_aWSKx--syBwMLdg)

#### 1. 为什么需要工厂模式
非工厂模式:  
   面向对象编程的方式, 一旦依赖了某个类的方法, 想要在换一套实现变得不可能. 所谓"面向扩展开放, 面向修改封闭"(如下, 不能修改Service中new JpaDao这行, 也不能直接修改JpaDao这个类)
   
```java
interface Dao{
    public void doSomething();
};
class JpaDao implements Dao{
    public void doSomething(){}
}

class Service{
    Dao dao = new JpaDao();
    public void service(){
        dao.doSomething();
    }
}
```

#### 2.简单工厂模式(静态工厂模式)  
工厂模式变为了面向接口编程, 简单工厂, 只是需要的对象提升成接口, 工厂模式下, 工厂类也有自己的接口   
**java的静态方法不能被覆盖, 重写**, 因此该工厂类不能被扩展
```java

interface Dao{
    public void doSomething();
};
class JpaDao implements Dao{
    public void doSomething(){}
}

class DaoFactory{  // 新增
    public static Dao getDao(){
        return new JpaDao();
    } 
}

class Service{
    Dao dao = DaoFactory.getDao();  // 修改
    public void service(){
        dao.doSomething();
    }
}
```

为了让工厂能够创建出来多个Dao,想到可以利用传入类名+反射的技巧, 来替代工厂类中直接new对象, 使其可以产生各种不同对象, 修改Factory  
```java
class DaoFactory{
    public static Dao getDao() throws ClassNotFoundException, IllegalAccessException, InstantiationException {
        Properties properties = new Properties();
        properties.load(this.getClass().getClassLoader().getResourceAsStream("config.prop"));
        String className = properties.getProperty("className");
        Class<?> daoClass = Class.forName((String) className);
        return (Dao)daoClass.newInstance();
    }
}
```
#### 3. 工厂模式
每个对象设置一个工厂类, 对象的生成由工厂完成, 让所有工厂类继承工厂接口
```java
interface Dao{
    public void doSomething();
};
class JpaDao implements Dao{ public void doSomething(){} }
class MybatisDao implements Dao{ public void doSomething(){} }
////////////////////////////////////////////////////////////////

interface DaoFactory{ public Dao getDao();}
class JpaFactory implements DaoFactory{
    @Override
    public Dao getDao() {
        return new JpaDao()
    }
}
class MybatisFactory implements DaoFactory{
    @Override
    public Dao getDao() {
        return new MybatisDao()
    }
}
/////////////////////////////////////////////////////////////////

class Service{
    private Dao dao;
    public void service(DaoFactory daoFactory){
        this.dao = daoFactory.getDao();
        this.dao.doSomething();
    }
}
```

#### 4. 抽象工厂模式 (可以产生同组对象的工厂模式)
```java
interface Dao{
    public void doSomething();
};
class JpaDao implements Dao{ public void doSomething(){} }
class MybatisDao implements Dao{ public void doSomething(){} }
////////////////////////////////////////////////////////////////

interface Hello{public void sayHello();}
class JpaHello implements Hello{ public void sayHello(){} }
class MybatisHello implements Hello{ public void sayHello(){} }
////////////////////////////////////////////////////////////////

interface DaoFactory{ public Dao getDao(); public Hello getHelo();}
class JpaFactory implements DaoFactory{
    @Override
    public Dao getDao() {
        return new JpaDao()
    }
    @Override
    public Hello getHelo() {
        return new JpaHello();
    }
}
class MybatisFactory implements DaoFactory{
    @Override
    public Dao getDao() {
        return new MybatisDao()
    }

    @Override
    public Hello getHelo() {
        return new MybatisHello();
    }
}
////////////////////////////////////////////////////////////////

class Service{
    private Dao dao;
    private Hello hello;
    public void service(DaoFactory daoFactory){
        this.dao = daoFactory.getDao();
        this.hello = daoFactory.getHelo();
        this.dao.doSomething();
        this.hello.sayHello();
    }
}

```